In [ ]:
# Installing
!pip install -q openai langchain gradio

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key")

In [ ]:
from langchain.chat_models import ChatOpenAI

# Initialize chat model
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name="gpt-3.5-turbo",
)

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a prompt template
template = """You are an expert travel consultant. You give specific and thoughtful advice to potential travelers."""

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [ ]:
from langchain.memory import ConversationBufferMemory

# Create conversation history store
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

In [ ]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Compose components in a chain using LCEL
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | chat_prompt
    | llm
)

In [ ]:
# Response function for ChatInterface
def stream_response(input, _):
    if input is not None:
        partial_message = ""
        for chunk in chain.stream({"input": input}): # LCEL streaming syntax
            partial_message += chunk.content
            yield partial_message

In [ ]:
import gradio as gr

gr.ChatInterface(
    stream_response, retry_btn=False, undo_btn=False, clear_btn=False
).queue().launch()

In [ ]:
# Prompt chaining example

from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate


# chain 1: Generating premises
comedian = "Kevin Hart"

template = """In bullet point form, wrie 5-10 potential topics for jokes about going to the movies that might be employed by {comedian}. Don't write the entire jokes just the premises. Mention the name of the comedian at the start of the bullet point\n"""
prompt_template = PromptTemplate(input_variables=["comedian"], template=template)
premises_chain = LLMChain(llm=llm, prompt=prompt_template)

# chain 2: Generate storyboard
template = """Here are a list of premises for jokes in the style of our requested comedian:
  {premises}
  From these bullet points, select 2-3 premises that could be combined into a funny joke, and note how they might flow with one and other.\n\n"""
prompt_template = PromptTemplate(input_variables=["premises"],template=template)
storyboard_chain = LLMChain(llm=llm, prompt=prompt_template)

# Chain 3: Create joke
template = """Using the following storyboard:
{storyboard}
  Please write a comedic joke in the style of our requested comedian.
  """
prompt_template = PromptTemplate(input_variables=["storyboard"],template=template)
final_joke_chain = LLMChain(llm=llm, prompt=prompt_template)
final_joke_chain_seq = SimpleSequentialChain(
      chains=[premises_chain, storyboard_chain, final_joke_chain],verbose=True
)

print(final_joke_chain_seq.run(comedian))

In [ ]:
"""
Final exercise:

1) Like with our travel consultant above, we would like to create a chatbot.

Rather than a travel consultant, we would like this chatbot to be a financial consultant of which the user can ask personal finance questions.

Also, we would like to create a Gradio interface with streamed responses.


2) Create a prompt chain to write a poem about McDonald's in the style of a user supplied poet (similar to the comedian joke chain we created above).

"""